In [11]:
import csv
import json
from copy import deepcopy
import ast
csv.field_size_limit(1131072)

1131072

In [12]:
source_data = []
with open('data/zh_test/source/to_tasa_all_zh_trial.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for item in reader:
        source_data.append(item)

In [13]:
print(source_data[0])

{'src_tokens': '[\'There\', \'you\', \'go\', \'.\', \'You\', "\'re", \'all\', \'done\', \'!\']', 'tar_tokens': "['就', '这', '样', '啦', '耳', '洞', '打', '好', '了', '!']", 'config_obj': '{"src_enable_retokenize": false, "version": {"PATCH": 0, "MAJOR": 1, "MINOR": 0}, "tar_enable_retokenize": false,"src_head_inds":[0],"alignment":[[false, true, true, false, false, false, false, false, false, false], [false, false, false, false, false, false, false, false, false, false], [false, false, false, false, false, false, false, false, false, false], [false, false, false, false, false, false, false, false, false, false], [false, false, false, false, false, false, false, false, false, false], [false, false, false, false, false, false, false, false, false, false], [false, false, false, false, false, false, false, false, false, false], [false, false, false, false, false, false, false, false, false, false], [false, false, false, false, false, false, false, false, false, false]]}', 'mention_id': 's09e03c09t

In [14]:
source_inputs = []
for set_key in ["0_1000", "1000_2000", "2000_3000", "3000_4000","4000_5000", "5000_6000","6000_7000", "7000_end"]:
    with open('data/zh_test/inputs/to_tasa_zh_drop_test_set_'+set_key+'.csv', 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for item in reader:
            source_inputs.append(item)

In [15]:
coe_annotations = []
for split_name in ["batch_0", "batch_1", "batch_2","batch_3","batch_4","batch_5","batch_6"]:
    with open('data/zh_test/annotations/'+split_name+'.csv', 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for item in reader:
            coe_annotations.append(item)

In [16]:
all_data = {}
with open('data/zh_test/source/all_coref_data_en_finalized.json', 'r') as f:
    temp = json.load(f)
    for item in temp:
        all_data[item['scene_id']] = item

mention_dict = {}
for item in all_data:
    annotations = all_data[item]['annotations']
    for anno in annotations:
        query = anno['query']
        antecedents = anno['antecedents']
        mention_dict[query['mention_id']] = [query['sentenceIndex'], query['startToken'],query['endToken']]
        if antecedents!=[]:
            if isinstance(antecedents[0], dict):
                for ante in antecedents:
                    mention_dict[ante['mention_id']] = [ante['sentenceIndex'], ante['startToken'],ante['endToken']]

In [17]:
all_data_zh = {}
with open('data/zh_test/source/all_coref_data_en_zh_finalized.json', 'r') as f:
    temp = json.load(f)
    for item in temp:
        all_data_zh[item['scene_id']] = item

mention_dict_zh = {}
for item in all_data_zh:
    annotations = all_data_zh[item]['annotations']
    for anno in annotations:
        query = anno['query']
        antecedents = anno['antecedents']
        mention_dict_zh[query['mention_id']] = [query['sentenceIndex'], query['startToken'],query['endToken']]
        if antecedents!=[]:
            if isinstance(antecedents[0], dict):
                for ante in antecedents:
                    mention_dict_zh[ante['mention_id']] = [ante['sentenceIndex'], ante['startToken'],ante['endToken']]

In [18]:
print(all_data_zh['s09e01c04t'])

{'sentences': [['所以', '一会', '听到', '的', '音乐', '就是', '你们', '的', '提示', '还有', '什么', '问题', '吗', '?'], ['呃', '我们', '购买', '的', '套装', '里', '好像', '还', '应该', '包括', '地毯', '上铺', '满', '玫瑰', '花瓣', '的'], ['玫瑰', '花瓣', '怎么', '没有', '了', '?'], ['那', '算了吧'], ['不过', '我们', '可以', '用', '洗手间', '里', '的', '干花', '!'], ['真的', '这样', '就行了'], [], [], ['我', '在', '前面', '等', '你们'], ['就是', '现在', '了', '是', '啊'], [], [], ['我', '也', '爱', '你'], ['你们', '还要', '不要', '厕所', '的', '干花', '了', '?'], ['不用', '谢谢', '!', '没事', '了', '!'], []], 'annotations': [{'query': {'sentenceIndex': 0, 'startToken': 4, 'endToken': 5, 'mention_id': 's09e01c04t|1'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}, {'query': {'sentenceIndex': 0, 'startToken': 6, 'endToken': 7, 'mention_id': 's09e01c04t|3'}, 'antecedents': ['null_projection']}, {'query': {'sentenceIndex': 0, 'startToken': 6, 'endToken': 9, 'mention_id': 's09e01c04t|5'}, 'antecedents': ['null_projection']}, {'query': {'sentenceIndex': 0, 'startToken': 10, 'endToken': 12,

In [19]:
def remove_drop_words(alignment):
    result = set()
    for start, end in alignment:
        result.add((start, end//2))
    return result

def extract_pair(result):
    starts = set()
    ends = set()
    for start, end in result:
        starts.add(start)
        ends.add(end)
    start_idx = min(list(starts))
    end_idx = max(list(ends))
    return [min(list(ends)), max(list(ends))+1]

def extract_alignment_pair_original(alignment):
    result = set()
    for s in range(len(alignment)):
        for t in range(len(alignment[0])):
            if alignment[s][t]:
                result.add((s, t))
    return sorted(list(result))

def extract_alignment_pair_space(alignment):
    result = set()
    for s in range(len(alignment)):
        for t in range(len(alignment[0])):
            if alignment[s][t]:
                result.add((s, t))
    return sorted(list(remove_drop_words(result)))

correction_dict = {}
remove_set = set()
add_dict = {}

# for i in range(len(coe_annotations)):
count = 0
# for i in range(6400):
for i in range(1000):
    extract_alignment_pair_original(json.loads(source_data[i]['config_obj'])['alignment'])
    extract_alignment_pair_space(json.loads(source_inputs[i]['config_obj'])['alignment'])

    if source_data[i]['mention_id'] in mention_dict_zh:
        temp_0 = extract_pair(extract_alignment_pair_original(json.loads(source_data[i]['config_obj'])['alignment']))
        temp_1 = extract_pair(extract_alignment_pair_space(json.loads(source_inputs[i]['config_obj'])['alignment']))
        # print(temp_0)
        # print(temp_1)
        sent_id, start, end = mention_dict_zh[source_data[i]['mention_id']]
        print(source_data[i]['mention_id'], source_data[i]['mention_id'][:10], mention_dict_zh[source_data[i]['mention_id']])
        print("".join(ast.literal_eval(source_data[i]['tar_tokens'])[temp_0[0]: temp_1[1]]))
        print("".join(all_data_zh[source_data[i]['mention_id'][:10]]['sentences'][sent_id][start: end]))
        print()

    original_result = extract_alignment_pair_space(json.loads(coe_annotations[i]['Input.config_obj'])['alignment'])
    corrected_result = extract_alignment_pair_space(json.loads(coe_annotations[i]['Answer.alignment']))
    if original_result==corrected_result:
        continue
    mention_id = source_data[i]['mention_id']
    # print(original_result, corrected_result)
    a = ()
    b = ()

    count += 1


    if original_result!=[]:
        a = extract_pair(original_result)
    if corrected_result!=[]:
        b = extract_pair(corrected_result)
    # print(source_data[i]['mention_id'], a, b)
    # print(mention_dict[mention_id], a)
    # print()

    # if source_data[i]['mention_id'] in mention_dict:
    #     mention_id = source_data[i]['mention_id']
    #     target = mention_dict[source_data[i]['mention_id']][1:]
    #     correction_dict[mention_id] = [a, b]
    if b==(): # Add mention_id to remove set
        remove_set.add(source_data[i]['mention_id'])
    elif a==(): # Collect mentions to add
        add_dict[source_data[i]['mention_id']] = [mention_dict[mention_id][0], b[0], b[1]]
    else:
        correction_dict[source_data[i]['mention_id']] = [mention_dict[mention_id][0], b[0], b[1]]

s09e03c09t|0 s09e03c09t [0, 1, 2]
这样
这样

s09e03c09t|3 s09e03c09t [0, 3, 4]
耳洞
耳洞

s09e03c09t|7 s09e03c09t [1, 8, 9]
我
我

s09e03c09t|9 s09e03c09t [1, 8, 10]
我妈妈
我妈妈

s09e03c09t|10 s09e03c09t [1, 11, 12]
我
我

s09e03c09t|12 s09e03c09t [1, 12, 13]
她
她

s09e03c09t|14 s09e03c09t [1, 15, 16]
我
我

s09e03c09t|16 s09e03c09t [1, 18, 20]
罪孽柜里
罪孽柜里

s09e03c09t|17 s09e03c09t [2, 0, 2]
你在
你在

s09e03c09t|18 s09e03c09t [2, 2, 3]
开玩笑
开玩笑

s09e03c09t|20 s09e03c09t [3, 4, 5]
玩笑
玩笑

s09e03c09t|21 s09e03c09t [3, 1, 2]
她
她

s09e03c09t|23 s09e03c09t [3, 5, 6]
我
我

s09e03c09t|25 s09e03c09t [3, 11, 12]
看电视
看电视

s09e03c09t|26 s09e03c09t [3, 9, 11]
柜子缝里
柜子缝里

s09e03c09t|27 s09e03c09t [4, 0, 1]
我
我

s09e03c09t|29 s09e03c09t [4, 4, 5]
你
你

s09e03c09t|31 s09e03c09t [4, 4, 8]
你和你妈妈
你和你妈妈

s09e03c09t|34 s09e03c09t [4, 6, 7]
你
你

s09e03c09t|36 s09e03c09t [4, 6, 8]
你妈妈
你妈妈

s09e03c09t|38 s09e03c09t [5, 1, 2]
她
她

s09e03c09t|40 s09e03c09t [5, 9, 10]
妈妈
妈妈

s09e03c09t|44 s09e03c09t [6, 0, 1]
你
你

s09e03c09t|46 s09e03c09t 

In [24]:
correction_dict = {}
remove_set = set()
add_dict = {}

# for i in range(len(coe_annotations)):
count = 0
# for i in range(6400):
for i in range(6400):
    extract_alignment_pair_original(json.loads(source_data[i]['config_obj'])['alignment'])
    extract_alignment_pair_space(json.loads(source_inputs[i]['config_obj'])['alignment'])

    if source_data[i]['mention_id'] in mention_dict_zh:
        temp_0 = extract_pair(extract_alignment_pair_original(json.loads(source_data[i]['config_obj'])['alignment']))
        temp_1 = extract_pair(extract_alignment_pair_space(json.loads(source_inputs[i]['config_obj'])['alignment']))
        # print(temp_0)
        # print(temp_1)
        sent_id, start, end = mention_dict_zh[source_data[i]['mention_id']]
        # print(source_data[i]['mention_id'], source_data[i]['mention_id'][:10], mention_dict_zh[source_data[i]['mention_id']])
        # print("".join(ast.literal_eval(source_data[i]['tar_tokens'])[temp_0[0]: temp_1[1]]))
        # print("".join(all_data_zh[source_data[i]['mention_id'][:10]]['sentences'][sent_id][start: end]))
        # print()

    original_result = extract_alignment_pair_space(json.loads(coe_annotations[i]['Input.config_obj'])['alignment'])
    corrected_result = extract_alignment_pair_space(json.loads(coe_annotations[i]['Answer.alignment']))
    if original_result==corrected_result:
        continue
    mention_id = source_data[i]['mention_id']
    # print(original_result, corrected_result)
    a = ()
    b = ()

    count += 1


    if original_result!=[]:
        a = extract_pair(original_result)
    if corrected_result!=[]:
        b = extract_pair(corrected_result)
    print(source_data[i]['mention_id'], a, b)
    print(mention_dict[mention_id], a)
    print()

    # if source_data[i]['mention_id'] in mention_dict:
    #     mention_id = source_data[i]['mention_id']
    #     target = mention_dict[source_data[i]['mention_id']][1:]
    #     correction_dict[mention_id] = [a, b]
    if b==(): # Add mention_id to remove set
        remove_set.add(source_data[i]['mention_id'])
    elif a==(): # Collect mentions to add
        add_dict[source_data[i]['mention_id']] = [mention_dict[mention_id][0], b[0], b[1]]
    else:
        correction_dict[source_data[i]['mention_id']] = [mention_dict[mention_id][0], b[0], b[1]]

s09e03c09t|0 [1, 3] ()
[0, 2, 3] [1, 3]

s09e03c09t|3 [4, 6] ()
[0, 6, 7] [4, 6]

s09e03c09t|6 () [1, 2]
[1, 2, 3] ()

s09e03c09t|17 [0, 2] ()
[2, 2, 3] [0, 2]

s09e03c09t|18 [2, 5] [3, 5]
[2, 4, 6] [2, 5]

s09e03c09t|25 [16, 19] [17, 19]
[3, 14, 16] [16, 19]

s09e03c09t|26 [12, 16] [14, 15]
[3, 17, 19] [12, 16]

s09e03c09t|42 () [2, 3]
[5, 9, 10] ()

s09e03c09t|62 [18, 20] [18, 19]
[8, 16, 17] [18, 20]

s09e03c09t|69 () [9, 10]
[9, 11, 12] ()

s09e03c09t|74 () [2, 3]
[10, 6, 7] ()

s09e03c09t|81 () [8, 9]
[11, 12, 13] ()

s09e03c09t|91 [5, 10] [7, 10]
[12, 7, 10] [5, 10]

s09e03c09t|103 [1, 5] [1, 3]
[13, 9, 13] [1, 5]

s09e03c09t|104 () [8, 9]
[13, 14, 15] ()

s09e03c09t|123 [16, 20] ()
[15, 13, 15] [16, 20]

s09e03c09t|141 [16, 17] [16, 18]
[19, 15, 16] [16, 17]

s09e03c09t|154 [14, 16] [13, 14]
[20, 15, 16] [14, 16]

s09e03c09t|171 () [3, 4]
[22, 5, 6] ()

s09e03c09t|177 () [15, 16]
[22, 15, 16] ()

s09e03c09t|178 [16, 24] [20, 24]
[22, 17, 23] [16, 24]

s09e03c09t|196 [9, 10] ()
[

KeyboardInterrupt: 

In [ ]:
print(count/len())

In [118]:
import pickle as pkl

scene_ids = set()
for item in correction_dict:
    scene_ids.add(item[:10])
for item in remove_set:
    scene_ids.add(item[:10])
for item in add_dict:
    scene_ids.add(item[:10])

all_results = {}
for item in scene_ids:
    all_results[item] = {
        "correction_dict": {},
        "remove_set": set(),
        "add_dict": {}
    }

for item in remove_set:
    all_results[item[:10]]['remove_set'].add(item)
for item in correction_dict:
    all_results[item[:10]]['correction_dict'][item] = correction_dict[item]
for item in add_dict:
    all_results[item[:10]]['add_dict'][item] = add_dict[item]

with open('data/zh/correction_results.pkl', 'wb') as f:
    pkl.dump(all_results, f)

In [119]:
print(all_results)

{'s09e09c07t': {'correction_dict': {'s09e09c07t|50': [8, 16, 19], 's09e09c07t|97': [13, 1, 3], 's09e09c07t|104': [13, 10, 13], 's09e09c07t|111': [14, 7, 9], 's09e09c07t|117': [15, 9, 12]}, 'remove_set': {'s09e09c07t|56', 's09e09c07t|31', 's09e09c07t|23', 's09e09c07t|5', 's09e09c07t|38', 's09e09c07t|39'}, 'add_dict': {'s09e09c07t|0': [0, 2, 3], 's09e09c07t|41': [8, 2, 4], 's09e09c07t|63': [8, 45, 46], 's09e09c07t|78': [10, 1, 2], 's09e09c07t|92': [11, 7, 8], 's09e09c07t|98': [13, 5, 6], 's09e09c07t|102': [13, 9, 10], 's09e09c07t|105': [13, 10, 11], 's09e09c07t|115': [15, 9, 10]}}, 's09e04c07t': {'correction_dict': {'s09e04c07t|13': [2, 15, 18], 's09e04c07t|17': [2, 9, 14], 's09e04c07t|38': [6, 14, 17]}, 'remove_set': {'s09e04c07t|70', 's09e04c07t|67'}, 'add_dict': {'s09e04c07t|11': [2, 9, 11], 's09e04c07t|27': [5, 7, 8], 's09e04c07t|36': [6, 14, 15], 's09e04c07t|57': [6, 30, 31], 's09e04c07t|83': [9, 13, 15]}}, 's09e10c04t': {'correction_dict': {'s09e10c04t|15': [3, 1, 2], 's09e10c04t|1

In [8]:
print(all_data.keys())
print(len(all_data))

dict_keys(['s01e01c00t', 's01e01c01t', 's01e01c03t', 's01e01c04t', 's01e01c05t', 's01e01c06t', 's01e01c07t', 's01e01c08t', 's01e01c09t', 's01e01c10t', 's01e03c00t', 's01e10c04t', 's01e03c01t', 's01e03c02t', 's01e03c03t', 's01e03c04t', 's01e03c05t', 's01e03c06t', 's01e03c07t', 's01e03c08t', 's01e03c09t', 's01e04c00t', 's01e04c01t', 's01e04c02t', 's01e04c03t', 's01e04c04t', 's01e04c05t', 's01e04c06t', 's01e04c07t', 's01e04c08t', 's01e04c09t', 's01e04c10t', 's01e04c11t', 's01e04c12t', 's01e05c00t', 's01e05c01t', 's01e05c02t', 's01e05c03t', '', 's01e05c05t', 's01e05c06t', 's01e05c07t', 's01e07c01t', 's01e07c02t', 's01e07c03t', 's01e07c04t', 's01e07c05t', 's01e09c00t', 's01e09c01t', 's01e09c02t', 's01e09c03t', 's01e09c04t', 's01e09c05t', 's01e09c06t', 's01e07c00t', 's01e09c07t', 's01e10c00t', 's01e10c01t', 's01e10c02t', 's01e10c03t', 's01e10c05t', 's01e10c07t', 's01e11c00t', 's01e11c01t', 's01e11c02t', 's01e11c03t', 's01e11c04t', 's01e11c05t', 's01e11c06t', 's01e11c07t', 's01e11c08t', 's04e

In [31]:
target = set()
for item in all_data.keys():
    if item[9]=="t":
        target.add(item[:6])
temp = [(int(item[1:3]), int(item[4:6])) for item in target]
print(sorted(list(target)))
print(len(target))
print(temp)

['s01e01', 's01e03', 's01e04', 's01e05', 's01e07', 's01e09', 's01e10', 's01e11', 's01e13', 's01e14', 's01e15', 's01e16', 's02e04', 's02e06', 's03e21', 's03e23', 's04e01', 's04e03', 's04e04', 's04e05', 's04e06', 's04e08', 's04e19', 's04e21', 's04e22', 's04e23', 's04e24', 's05e02', 's05e06', 's05e12', 's05e14', 's05e15', 's05e20', 's05e23', 's06e02', 's06e03', 's06e04', 's06e05', 's06e06', 's06e07', 's06e08', 's06e09', 's06e14', 's06e15', 's06e16', 's06e17', 's06e19', 's06e20', 's06e21', 's07e01', 's07e02', 's07e06', 's07e10', 's07e11', 's07e15', 's07e16', 's07e17', 's07e18', 's07e19', 's07e20', 's07e22', 's07e23', 's08e01', 's08e02', 's08e03', 's08e06', 's08e09', 's08e11', 's08e12', 's08e15', 's08e16', 's08e17', 's08e18', 's08e19', 's08e20', 's08e21', 's08e22', 's08e23', 's08e24', 's09e01', 's09e03', 's09e04', 's09e07', 's09e09', 's09e10', 's09e11', 's09e12', 's09e16']
88
[(7, 15), (6, 16), (7, 23), (5, 2), (9, 3), (5, 14), (1, 9), (6, 19), (7, 2), (1, 1), (8, 19), (3, 21), (7, 16), (8,

In [32]:
target = set()
for item in all_data.keys():
    if item[9]=="f":
        target.add(item[:6])
temp = [(int(item[1:3]), int(item[4:6])) for item in target]
print(sorted(list(target)))
print(len(target))
print(temp)

['s01e01', 's01e02', 's01e03', 's01e04', 's01e06', 's01e07', 's01e08', 's01e09', 's01e10', 's01e12', 's01e13', 's01e14', 's01e15', 's01e16', 's01e17', 's01e18', 's01e19', 's01e20', 's01e22', 's01e24', 's06e09']
21
[(1, 10), (1, 22), (1, 15), (1, 9), (1, 1), (1, 12), (1, 17), (1, 4), (1, 24), (1, 6), (1, 19), (1, 8), (1, 18), (1, 16), (1, 2), (6, 9), (1, 13), (1, 14), (1, 3), (1, 7), (1, 20)]


In [9]:
print(correction_dict.keys())

dict_keys(['s09e03c09t|0', 's09e03c09t|3', 's09e03c09t|17', 's09e03c09t|18', 's09e03c09t|25', 's09e03c09t|26', 's09e03c09t|62', 's09e03c09t|91', 's09e03c09t|103', 's09e03c09t|123', 's09e03c09t|141', 's09e03c09t|154', 's09e03c09t|178', 's09e03c09t|196', 's09e03c09t|197', 's09e03c09t|203', 's09e03c09t|217', 's09e03c09t|230', 's09e03c09t|234', 's09e03c09t|252', 's09e03c09t|267', 's09e03c09t|268', 's09e03c09t|270', 's09e03c10t|11', 's09e03c10t|12', 's09e03c10t|13', 's09e03c10t|16', 's09e03c10t|33', 's09e03c10t|34', 's09e03c10t|40', 's09e03c10t|41', 's09e03c10t|42', 's09e03c10t|43', 's09e03c10t|48', 's09e03c10t|57', 's09e03c10t|69', 's09e03c10t|70', 's09e03c10t|71', 's09e03c10t|72', 's09e03c11t|1', 's09e03c11t|10', 's09e03c11t|11', 's09e03c11t|14', 's09e03c11t|21', 's09e03c11t|28', 's09e03c11t|63', 's09e03c11t|71', 's09e03c12t|10', 's09e03c12t|28', 's09e03c12t|39', 's09e03c12t|49', 's09e03c12t|57', 's09e03c12t|58', 's09e03c12t|59', 's09e03c12t|88', 's09e03c12t|119', 's09e03c12t|120', 's09e0

In [10]:
print(all_data['s09e03c09t'])

{'sentences': [['就', '这', '样', '啦', '耳', '洞', '打', '好', '了', '!'], ['感', '觉', '挺', '不', '错', '的', '好', '漂', '亮', '天', '啊', '要', '是', '我', '妈', '妈', '看', '到', '我', '她', '要', '把', '我', '锁', '进', '罪', '孽', '柜', '里', '面', '壁', '思', '过', '了'], ['你', '在', '开', '玩', '笑', '吧', '?'], ['说', '实', '话', '她', '才', '是', '玩', '笑', '我', '还', '能', '从', '柜', '子', '缝', '里', '看', '电', '视', '呢'], ['我', '终', '于', '明', '白', '为', '什', '么', '你', '和', '你', '妈', '妈', '的', '关', '系', '不', '亲', '密', '了'], ['但', '是', '她', '要', '是', '结', '婚', '了', '她', '还', '是', '会', '告', '诉', '妈', '妈', '的'], ['你', '抓', '着', '这', '事', '不', '放', '手', '了', '吗', '?'], ['他', '们', '不', '知', '道', '这', '事', '让', '人', '很', '伤', '心', '我', '是', '说', '他', '们', '都', '爱', 'L', 'e', 'o', 'n', 'a', 'r', 'd'], ['好', '吧', '你', '们', '要', '我', '打', '电', '话', '吗', '?', '我', '打', '电', '话', '为', '什', '么', '我', '会', '这', '么', '紧', '张', '?'], ['这', '可', '以', '理', '解', 'A', 'm', 'y', '告', '诉', '妈', '妈', '她', '和', 'S', 'h', 'e', 'l', 'd', 'o', 'n', '分', '手', '

In [12]:
# Perform Correction
with open('data/zh/source/all_coref_data_en_zh_finalized.json', 'r') as f:
    source = json.load(f)

In [29]:
output = []
for i, scene in enumerate(source):
    if scene['scene_id'] != "s09e07c07t":
        continue
    new_annotations = []
    annotations = deepcopy(scene['annotations'])
    for item in annotations:
        query = deepcopy(item['query'])
        antecedents = deepcopy(item['antecedents'])
        # print(query, antecedents)
        # Correct Query
        if query['mention_id'] in correction_dict:
            print(query, correction_dict[query['mention_id']])

        new_annotations.append({
            "query": query,
            "antecedents": antecedents
        })



{'sentenceIndex': 1, 'startToken': 0, 'endToken': 1, 'mention_id': 's09e07c07t|2'} [[0, 1], [15, 16]]
{'sentenceIndex': 1, 'startToken': 43, 'endToken': 45, 'mention_id': 's09e07c07t|7'} [[43, 45], ()]
{'sentenceIndex': 2, 'startToken': 1, 'endToken': 4, 'mention_id': 's09e07c07t|8'} [[1, 4], ()]
{'sentenceIndex': 2, 'startToken': 4, 'endToken': 6, 'mention_id': 's09e07c07t|10'} [[4, 6], ()]
{'sentenceIndex': 4, 'startToken': 8, 'endToken': 10, 'mention_id': 's09e07c07t|13'} [[8, 10], ()]
{'sentenceIndex': 5, 'startToken': 8, 'endToken': 9, 'mention_id': 's09e07c07t|19'} [[8, 9], [17, 18]]
{'sentenceIndex': 7, 'startToken': 3, 'endToken': 4, 'mention_id': 's09e07c07t|30'} [[3, 4], ()]
{'sentenceIndex': 7, 'startToken': 3, 'endToken': 4, 'mention_id': 's09e07c07t|30'} [[3, 4], ()]
{'sentenceIndex': 7, 'startToken': 10, 'endToken': 12, 'mention_id': 's09e07c07t|33'} [[10, 12], ()]
{'sentenceIndex': 8, 'startToken': 3, 'endToken': 5, 'mention_id': 's09e07c07t|44'} [[3, 5], ()]
{'sentenceI

In [30]:
print(correction_dict['s09e07c07t|30'])

[[3, 4], ()]


In [31]:
print(all_data['s09e07c07t'])

{'sentences': [[], ['你', '在', '这', '里', '感', '觉', '还', '好', '嘛', '?', '没', '事', '挺', '好', '的', '我', '感', '觉', '自', '己', '像', '个', '考', '古', '学', '家', '像', '是', 'I', 'n', 'd', 'i', 'a', 'n', 'a', 'J', 'o', 'n', 'e', 's', '与', '独', '栋', '别', '墅', '嘿', '看', '啊', '!'], ['我', '捡', '了', '个', '贝', '壳'], [], ['其', '实', '是', '老', '鼠', '的', '骨', '头', '淡', '定'], ['这', '的', '灰', '尘', '够', '多', '的', '了', '你', '就', '别', '再', '折', '腾', '了', '抱', '歉', '你', '知', '道', '.', '.', '.', '这', '里', '是', '什', '么', '位', '置', '吗', '?'], ['E', 'M', 'P', 'T', 'Y'], ['大', '概', '吧', '我', '觉', '得', '那', '边', '是', '小', '客', '厅'], ['好', '吧', '那', '这', '里', '就', '是', '餐', '厅', '了', '没', '错'], ['不', '过', '把', '小', '客', '厅', '改', '造', '成', '婴', '儿', '室', '也', '不', '会', '很', '费', '事'], ['我', '为', '什', '么', '会', '不', '想', '听', '你', '说', '这', '件', '事', '?'], ['因', '为', 'B', 'e', 'r', 'n', 'i', 'e', '说', '你', '不', '想', '要', '孩', '子', '不', '是', '这', '样', '的', '我', '.', '.', '.', '我', '很', '希', '望', '她', '能', '怀', '上', '一', '个'

In [88]:
sample = all_data['s09e03c10t']
sentences = sample['sentences']
annotations = sample['annotations']
for sent in sentences:
    print(sent)

[]
['这', '个', '叫', '做', '拉', '玛', '泽', '呼', '吸', '可', '以', '帮', '你', '用', '力']
['那', '你', '用', '子', '宫', '推', '好', '了']
['应', '该', '有', '个', '更', '容', '易', '的', '办', '法', '把', '它', '弄', '下', '来', '的']
['是', '的', '这', '不', '过', '就', '是', '个', '物', '理', '学', '问', '题', '罢', '了']
['阿', '基', '米', '德', '曾', '经', '说', '了', '"', '给', '我', '一', '根', '足', '够', '长', '的', '的', '杠', '杆', '和', '一', '个', '支', '点', '我', '就', '能', '撬', '动', '地', '球', '"']
['我', '们', '只', '需', '要', '找', '点', '原', '材', '料', '把', '这', '个', '杠', '杆', '做', '出', '来', '就', '行', '了']
['一', '点', '不', '错', '我', '敢', '打', '赌', '这', '附', '近', '有', '东', '西', '能', '当', '这', '个', '"', '撬', '棍', '"']
['让', '我', '找', '找', '"', '撬', '棍', '"', '在', '哪', '呢', '.', '.', '.']
['大', '家', '等', '一', '下', '!', '我', '们', '按', '顺', '序', '一', '个', '个', '解', '决', '问', '题', '首', '先', '我', '们', '得', '决', '定', '我', '们', '是', '叫', '它', '杠', '杆', '还', '是', '撬', '棍', '等', '我', '们', '决', '定', '叫', '它', '杠', '杆', '之', '后', '就', '可', '以', '卷', '起', '袖', '子'

In [89]:
for item in annotations:
    print(item)
    print(sentences[item['query']['sentenceIndex']][item['query']['startToken']: item['query']['endToken']])
    print()

{'query': {'sentenceIndex': 1, 'startToken': 0, 'endToken': 2, 'mention_id': 's09e03c10t|3'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['这', '个']

{'query': {'sentenceIndex': 1, 'startToken': 12, 'endToken': 13, 'mention_id': 's09e03c10t|6'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['你']

{'query': {'sentenceIndex': 2, 'startToken': 1, 'endToken': 2, 'mention_id': 's09e03c10t|7'}, 'antecedents': ['Raj']}
['你']

{'query': {'sentenceIndex': 2, 'startToken': 3, 'endToken': 5, 'mention_id': 's09e03c10t|11'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['子', '宫']

{'query': {'sentenceIndex': 3, 'startToken': 2, 'endToken': 4, 'mention_id': 's09e03c10t|12'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['有', '个']

{'query': {'sentenceIndex': 3, 'startToken': 5, 'endToken': 13, 'mention_id': 's09e03c10t|13'}, 'antecedents': ['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']}
['容', '易', '的', '办', 